<a href="https://colab.research.google.com/github/bailos/D.I.T/blob/main/Lab_Methode_Lasso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Definition**  
#### **La Regularisation**
En termes simples, **la régularisation** est utilisée pour éviter le sur-ajustement en limitant la capacité d’apprentissage ou la flexibilité d’un modèle d’apprentissage machine.  

Il existe deux types de techniques de régularisation de base : **la régression Ridge (pénalisation L2)** ou **la régression Lasso (pénalisation L1)**. Une autre méthode combinant les deux précédentes est la méthode **ElasticNet**.  

La régression Lasso est assez similaire à la régression Ridge, mais avec une différence.
1. Les deux ajoutent une pénalité aux coefficients non nuls.
2. La régression Ridge pénalise la somme des coefficients au carré
3. La régression de Lasso pénalise la somme de leurs valeurs absolues
$$
L_{\text {classic regression }}(\hat{\beta})=\sum_{i=1}^n\left(Y_i-\hat{Y}_i\right)^2
$$
$$
L_{\text {lasso }}(\hat{\beta})=\sum_{i=1}^n\left(Y_i-\hat{Y}_i\right)^2+\lambda * \sum_{j=1}^m\left|\hat{\beta}_j\right|
$$
$$
L_{\text {ridge }}(\hat{\beta})=\sum_{i=1}^n\left(Y_i-\hat{Y}_i\right)^2+\lambda * \sum_{j=1}^m \hat{\beta}_j^2
$$

La **régression au LASSO (Least Absolute Shrinkage and Selection Operator)** est une technique de régularisation. Il est utilisé sur les méthodes de régression pour une prédiction plus précise et d'éviter les cas d'overfitting (sur-ajustement) ou d'underfitting (sous-ajustement).
La différence entre ces méthodes se situe sur le coefficient de pénalisation $\lambda$ qu'elle applique à la fonction coût de la régression linéaire classique.  

**$\lambda:$** contrôle l'intensité de la pénalité.

**$\lambda$ = 0** , Régression Lasso = Régression linéaire  

*Au fur et à mesure que λ augmente, de plus en plus de coefficients sont mis à zéro et éliminés (théoriquement, lorsque λ = ∞ tous les coefficients sont éliminés).*

#### **Notions utulisés**
- **Biais**: Il s'agit en fait du taux d'erreur des données d'apprentissage. Lorsque le taux d'erreur a une valeur élevée, nous l'appelons High Bias et lorsque le taux d'erreur a une valeur faible, nous l'appelons Low Bias.
- **Variance** : La différence entre le taux d'erreur des données d'apprentissage et des données de test est appelée variance. Si la différence est élevée, on parle de variance élevée et lorsque la différence d'erreurs est faible, on parle de variance faible. Habituellement, nous voulons faire une faible variance pour généraliser notre modèle.
- **Underfitting**: Underfitting fait référence à un modèle qui ne peut ni bien fonctionner sur les données de formation ni se généraliser à de nouvelles données. Voici quelque raison de ce phenomène:    
  1. Biais élevé et faible variance.
  2. La taille de l'ensemble de données d'entraînement utilisé n'est pas suffisante.
  3. Le modèle est trop simple.
  4. Les données d'entraînement ne sont pas nettoyées et contiennent également du bruit.
- **Overfitting**: Un modèle statistique est dit surajusté lorsque le modèle ne fait pas de prédictions précises sur les données de test mais fait un très bon apprentissage. Ceci est causé parfois par:  
  1. Une Forte variance et un faible biais.
  2. Le modèle est trop complexe.
  3. La taille des données d'entraînement est petite

## **Mise en Pratique**

Nous importons les bibliothèques nécessaires,y compris les modules Lasso de scikit-learn pour la régression Lasso, train_test_split pour séparer les données en ensembles d'entraînement et de test StandardScaler pour normaliser les données, et mean_squared_error et r2_score pour évaluer le modèle.

In [3]:
# Importer les bibliothèques nécessaires
import pandas as pd 
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Nous chargeons les données et les séparons en variables d'entrée X et variable de sortie y.

In [5]:
# Charger les données
data = pd.read_csv('/content/drive/MyDrive/Master_IA_DIT_2022/Machine learning/diabetes.csv')
data.head(5)
Y = data['Outcome']
X = data.drop(['Outcome'], axis=1)
X


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63
764,2,122,70,27,0,36.8,0.340,27
765,5,121,72,23,112,26.2,0.245,30
766,1,126,60,0,0,30.1,0.349,47


# New Section

Nous utilisons train_test_split pour diviser les données en ensembles d'entraînement et de test, avec une taille de test de 20%.

In [6]:
# Séparer les données en ensembles d'entraînement et de test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

Nous normalisons les données en utilisant StandardScaler pour mettre les variables sur la même échelle.

In [7]:
# Normaliser les données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Nous créons un objet Lasso avec un paramètre alpha de 0.1, qui contrôle l'intensité de la pénalisation L1.

In [8]:
# Créer un modèle Lasso
lasso = Lasso(alpha=0.1)

Nous entraînons le modèle en utilisant les données d'entraînement normalisées.

In [9]:
# Entraîner le modèle
lasso.fit(X_train_scaled, Y_train)

Lasso(alpha=0.1)

Nous utilisons le modèle entraîné pour prédire les valeurs sur les données de test.

In [10]:
# Prédire les valeurs
Y_pred = lasso.predict(X_test_scaled)

Nous évaluons le modèle en calculant l'erreur quadratique moyenne MSE et le coefficient de détermination R² entre les valeurs prédites et les valeurs réelles.

In [11]:
# Évaluer le modèle
mse = mean_squared_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)

In [12]:
# Afficher les résultats
print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 0.18430487893995587
R-squared: 0.19724986061708094
